In [31]:
import pandas as pd
article = pd.read_excel('Articles56.xlsx')
daily = pd.read_excel('Dailies56.xlsx')
label = pd.read_excel('label.xlsx')

article = article[['id','title','description','content']]
daily = daily[['id','title','content']]
article.name='article'
daily.name='daily'

import re
def clean(raw):
    r1 = u'[a-zA-Z0-9’!"#$%&\'()*+,-./:;<=>?@，。?★、…【】《》？“”‘’！[\\]^_`{|}~\s（）：△ ©]+'
    r2 = "[\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）]+"
    r3 =  "[.!//_,$&%^*()<>+\"'?@#-|:~{}]+|[——！\\\\，。=？、：“”‘’《》【】￥……（）△:]+" 
    r4 =  "\\【.*?】+|\\《.*?》+|\\#.*?#+|[.!/_,$&%^*()<>+""'?@|:~{}#]+|[——！\\\，。=？、：“”‘’￥……（）《》【】]"

    text1 = re.sub(r1,'',raw)
    text2 = re.sub(r2,'',text1)
    text3 = re.sub(r3,'',text2)
    text4 = re.sub(r4,'',text3)
    return text4


def septext(text):
    Lword = []
    for x in text: 
        Lword.append(''.join(x))
    with open('/home/lene/Documents/chinese_stop_words.txt') as f:
        stopWords = [line.strip() for line in f.readlines()]
    stopWords.extend(['跟', '后', '在','再次','多个','的','另','被','年','月','而','与','所以','这些', \
                      '近年来','我','文曾','至','以','去年','为','已','人'])
    words = [w for w in Lword if w not in stopWords]
    return words

import jieba
import warnings
warnings.filterwarnings('ignore')

def token(df):
    df['ccontent'] = ''
    df['ctitle'] = ''
    df.content = df.content.astype(str)
    df.title = df.title.astype(str)

    for i in range(len(df)):
        df.ccontent[i] = septext(jieba.cut(clean(df.content[i])))
        df.ctitle[i] = septext(jieba.cut(clean(df.title[i])))
    return df

from ast import literal_eval
taga = pd.read_excel('article_tag_hecate.xlsx')
taga =tag[['title','name']]
tagd = pd.read_excel('daily_tag_hecate.xlsx')
tagd =tagd[['title','name']]

def merge(df,tag):
    df1 = pd.merge(df,tag,on='title',how='left')
    return df1

def transfername(df):
    df['tag_title'] = ''
    df['tag_name'] = ''
    df['tag_content'] = ''
    df.ctitle = df.ctitle.apply(set)
    df.ccontent = df.ccontent.apply(set)
    for i in range(len(df)):
        if str(df['name'][i])!='nan':
            df['name'][i] = set(literal_eval(df['name'][i]))
    return df

def match(words):
    tag=set()
    for column in label.columns:
        for word in words:
            if word in label[column].tolist():
                tag.add(str(column))
    return tag

def taged(df,base,tag):
    for i in range(len(df)):
        if str(df[base][i])!='nan':
            df[tag][i] = match(df[base][i])
    return df

def generate(df,tag):
    df1 = df.dropna(subset=['title'])
    df1 = merge(token(df1),tag)
    df1 = transfername(df1)
    taged(df1,'ctitle','tag_title')
    taged(df1,'name','tag_name')
    taged(df1,'ccontent','tag_content')
    df1.to_excel(df.name+'_tagged.xlsx')

# generate(daily,tagd)
# generate(article,tagd)

In [123]:
import json
eigen = pd.read_excel('Eigen56.xlsx')
eigen = eigen[['id','source']]
en = eigen[eigen.source.map(lambda x:x.startswith('{"desc":'))]
ch = eigen[eigen.source.map(lambda x:x.startswith('{"url":'))]
en['source'] = en['source'].apply(lambda x:json.loads(x))
en['content'] = en['source'].apply(lambda x:x['desc'])
en['title'] = en['source'].apply(lambda x:x['title'])
en = en.drop('source',1)

In [79]:
en.to_excel('eigen_en.xlsx')

In [124]:
ch = ch.reset_index(drop=True)

In [108]:
ch['source'][0]

'{"url": "http://new.qq.com/omn/20190506/20190506A0C1SI.html", "author": "人民网", "source": "NewsQQSpiderItem", "content": "\\n利用视觉辅助仪器读报纸。(图片来源：日本《朝日新闻》网站) \\n人民网东京5月6日电 据《朝日新闻》网站报道，日本三重县松坂市视觉残疾人协会日前在松坂市福利会馆内举办了一场具有划时代意义的视觉辅助仪器体验会。该仪器内置 AI(人工智能)技术，通过小型摄像机可将识别到的文字等转化为声音。除了盲人之外，还有40多名与行政和福祉相关的工作人员参加。大家对这一最新技术都非常赞叹。 \\n利用视觉辅助仪器识别纸钞。(图片来源：日本《朝日新闻》网站) \\n该仪器是由以色列一家企业开发出来的，去年9月又开发出可识别日语文字的产品。将该仪器安装在眼镜框上，指着你想知道的东西即可让摄像机将捕捉到的文字、人脸、纸币、颜色和时刻等识别出来，随后将信息转换为声音用耳边的扬声器代为传达。该仪器可以储存100张人脸的信息，它可以告知你面前的人是谁。 \\n体验者戴着安装有该仪器的眼镜，把手里的传单和报纸放在脸前阅读。虽然有时候也难以把想要阅读的文章指给摄像机，但只要触摸仪器，就可以拍摄。因此很多人笑着说:“这款机器很好用”“只要用习惯了，感觉会很方便”。(编译:于舒婷 审稿:陈建军)", "pubdate": "2019-05-06T16:16:00Z", "category": "正例", "headline": "日本推出一款视觉辅助仪器 利用AI技术可将文字转化为声音", "keywords": ["ai技术", "视觉", "日本", "陈建军", "摄像机", "识别", "人脸"], "articleid": "NewsQQSpiderItem/20190506A0C1SI", "keysentences": null, "thumbnailurl": "https://inews.gtimg.com/newsapp_ls/0/8851892044_640330/0"}'

In [125]:
for i in range(len(ch)):
    try:
        ch['source'][i] = json.loads(ch['source'][i])
    except:
        print(ch['id'][i])
        pass

1173656
1173726


In [128]:
ch1 = ch[ch['id']!=1173656]
ch1 = ch1[ch1['id']!=1173726]
ch1['content'] = ch1['source'].apply(lambda x:x['content'])
ch1['keyword'] = ch1['source'].apply(lambda x:x['keywords'])

In [143]:
ch1['title'] = ch1['source'].apply(lambda x:x['headline'])

In [ ]:
ch1.drop('source',1,inplace=True)

In [162]:
ch1.to_excel('eigen_ch.xlsx')

In [145]:
ch1.head()

,id,source,content,keyword,title
0,1169528,{'url': 'http://new.qq.com/omn/20190506/201905...,\n利用视觉辅助仪器读报纸。(图片来源：日本《朝日新闻》网站) \n人民网东京5月6日电 据...,"[ai技术, 视觉, 日本, 陈建军, 摄像机, 识别, 人脸]",日本推出一款视觉辅助仪器 利用AI技术可将文字转化为声音
1,1169529,{'url': 'https://www.jiqizhixin.com/articles/2...,虽然遗传学和现代影像学已经取得重大进展，但诊断结果还是会令大部分乳腺癌患者感到震惊。对一些患...,"[模型, 乳腺癌, 深度学习, 混淆矩阵, 电子病历]",早发现早治疗系列之乳腺癌：DL模型为患者争取五年治疗时间
2,1169530,{'url': 'https://www.yicai.com/news/100184933....,\n想知道最近该不该跳槽？ \n近日，全球职场社交平台LinkedIn（领英）携手全球知名人...,"[第一财经, 人才, 数字经济, 互联网公司, 领英, 企业, 人才流动, 人力资本]",数字经济时代，中国职场人除了互联网公司还能去哪儿？
3,1169531,{'url': 'https://www.yicai.com/news/100184927....,\n“没有典型场景应用没有意义，没有超越英伟达的芯片没有意义，没有世界级的算法没有意义。”依...,"[依图, ai芯片, 芯片, 依图科技, 第一财经, 英伟达, 算法, 人工智能技术, 推理...",对标巨头英伟达，依图科技发布云端AI芯片
4,1169532,{'url': 'https://www.jiqizhixin.com/articles/2...,前言爱奇艺HomeAI服务于iQIYI内的多个平台和产品，提供以视频点播为中心的智能语音交互...,"[语言模型, 语音识别, 语音交互, 模型, 爱奇艺, 语音助手, 识别]",爱奇艺HomeAI在语音交互领域的探索


In [147]:
ch1 = ch1.rename(columns={'keyword':'name'})

In [148]:
ch1.name = ch1.name.apply(set)

In [150]:
ch1.drop('source',1,inplace=True)

In [155]:
ch1 = ch1.drop_duplicates(subset=['title']).reset_index(drop=True)

In [157]:
token(ch1)

,id,content,name,title,ccontent,ctitle
0,1169528,\n利用视觉辅助仪器读报纸。(图片来源：日本《朝日新闻》网站) \n人民网东京5月6日电 据...,"{识别, ai技术, 陈建军, 视觉, 人脸, 日本, 摄像机}",日本推出一款视觉辅助仪器 利用AI技术可将文字转化为声音,"[利用, 视觉, 辅助, 仪器, 读, 报纸, 图片, 来源, 日本, 朝日新闻, 网站, ...","[日本, 推出, 一款, 视觉, 辅助, 仪器, 利用, 技术, 可, 将, 文字, 转化,..."
1,1169529,虽然遗传学和现代影像学已经取得重大进展，但诊断结果还是会令大部分乳腺癌患者感到震惊。对一些患...,"{混淆矩阵, 乳腺癌, 深度学习, 模型, 电子病历}",早发现早治疗系列之乳腺癌：DL模型为患者争取五年治疗时间,"[虽然, 遗传学, 和, 现代, 影像学, 已经, 取得, 重大进展, 但, 诊断, 结果,...","[早, 发现, 早, 治疗, 系列, 之, 乳腺癌, 模型, 患者, 争取, 五年, 治疗,..."
2,1169530,\n想知道最近该不该跳槽？ \n近日，全球职场社交平台LinkedIn（领英）携手全球知名人...,"{数字经济, 人力资本, 人才流动, 企业, 第一财经, 互联网公司, 人才, 领英}",数字经济时代，中国职场人除了互联网公司还能去哪儿？,"[想, 知道, 最近, 该不该, 跳槽, 近日, 全球, 职场, 社交, 平台, 领英, 携...","[数字, 经济, 时代, 中国, 职场, 除了, 互联网, 公司, 还, 能, 去, 哪儿]"
3,1169531,\n“没有典型场景应用没有意义，没有超越英伟达的芯片没有意义，没有世界级的算法没有意义。”依...,"{计算机视觉, 推理, 第一财经, 芯片, 算法, 依图科技, 英伟达, ai芯片, 人工智...",对标巨头英伟达，依图科技发布云端AI芯片,"[没有, 典型, 场景, 应用, 没有, 意义, 没有, 超越, 英, 伟达, 芯片, 没有...","[对标, 巨头, 英, 伟达, 依图, 科技, 发布, 云端, 芯片]"
4,1169532,前言爱奇艺HomeAI服务于iQIYI内的多个平台和产品，提供以视频点播为中心的智能语音交互...,"{识别, 语言模型, 语音交互, 语音识别, 模型, 爱奇艺, 语音助手}",爱奇艺HomeAI在语音交互领域的探索,"[前言, 爱奇艺, 服务, 于, 内, 平台, 和, 产品, 提供, 视频点播, 中心, 智...","[爱奇艺, 语音, 交互, 领域, 探索]"
5,1169533,\n第三届世界智能大会还有10天就开幕了，16日至19日在天津梅江会展中心与大家见面的第三届...,"{智能驾驶, 应用, 大会, 自动驾驶汽车, 挑战赛, 天津, 智能, 智慧城市, 智慧生活...",第三届世界智能大会将于本月16日至19日在津举行,"[第三届, 世界, 智能, 大会, 还有, 天, 就, 开幕, 了, 日, 日, 天津, 梅...","[第三届, 世界, 智能, 大会, 将, 于, 本月, 日至, 日, 津, 举行]"
6,1169534,\n美国当地时间 4 月 29 日，知名玩具机器人 Vector 的制造商 Anki 宣布倒...,"{产品, 玩具, 语音交互, 交互, 机器人, 语音助手, 苹果, 扫地机器人}",苹果发布会上的机器人公司宣布倒闭，他们可能只是来早了8个世纪,"[美国, 当地, 时间, 日, 知名, 玩具, 机器人, 制造商, 宣布, 倒闭, 将, 遣...","[苹果, 发布会, 上, 机器人, 公司, 宣布, 倒闭, 他们, 可能, 只是, 来, 早..."
7,1169535,\n消费科技快速发展，企业必须持续变革，否则就有被淘汰的危险——初创企业尤其如此。当然，如果...,"{智能手表, 出门问问, 企业, 智能耳机, 语音助理, 独角兽, 问问魔镜, 人工智能, 谷歌}",独角兽出门问问重拾AI愿景，WHY？,"[消费, 科技, 快速, 发展, 企业, 必须, 持续, 变革, 否则, 就, 有, 淘汰,...","[独角兽, 出门, 问问, 重拾, 愿景]"
8,1169536,"在位于德州达拉斯以北约 40 公里的小城弗里斯克（Frisco, TX），一辆行驶在路面上的...","{吴恩达, 打车, 王弢, 司机, 无人驾驶, 车辆, 自动驾驶, 车}",Drive.ai无人打车服务进德州，我们用15分钟见识了这位「新手驾驶员」,"[位于, 德州, 达拉斯, 北约, 公里, 小城, 弗里斯, 克, 一辆, 行驶, 路面, ...","[无人, 打车, 服务, 进, 德州, 我们, 用, 分钟, 见识, 了, 这位, 「, 新..."
9,1169537,上个月，微软代季峰等研究者发布的一篇论文提出了一种可变形卷积网络，该研究「引入了两种新的模块...,"{可变形卷积网络, 机器之心, 代码}",微软发布可变形卷积网络代码：可用于多种复杂视觉任务,"[上个月, 微软, 代季峰, 等, 研究者, 发布, 一篇, 论文, 提出, 了, 一种, ...","[微软, 发布, 可, 变形, 卷积, 网络, 代码, 可, 用于, 多种, 复杂, 视觉,..."


In [160]:
ch1['tag_title'] = ''
ch1['tag_name'] = ''
ch1['tag_content'] = ''
ch1.ctitle = ch1.ctitle.apply(set)
ch1.ccontent = ch1.ccontent.apply(set)

In [161]:
ch1

,id,content,name,title,ccontent,ctitle,tag_title,tag_name,tag_content
0,1169528,\n利用视觉辅助仪器读报纸。(图片来源：日本《朝日新闻》网站) \n人民网东京5月6日电 据...,"{识别, ai技术, 陈建军, 视觉, 人脸, 日本, 摄像机}",日本推出一款视觉辅助仪器 利用AI技术可将文字转化为声音,"{报纸, 到, 审稿, 福利, 转化, 福祉, 是, 体验, 这款, 你, 知道, 传达, ...","{声音, 一款, 仪器, 利用, 将, 转化, 可, 视觉, 技术, 推出, 辅助, 文字,...",,,
1,1169529,虽然遗传学和现代影像学已经取得重大进展，但诊断结果还是会令大部分乳腺癌患者感到震惊。对一些患...,"{混淆矩阵, 乳腺癌, 深度学习, 模型, 电子病历}",早发现早治疗系列之乳腺癌：DL模型为患者争取五年治疗时间,"{×, 混合, 同样, 供应商, 起来, 分配, 最低, 太迟, 一些, 重要性, 此外, ...","{之, 时间, 早, 乳腺癌, 争取, 五年, 治疗, 模型, 系列, 发现, 患者}",,,
2,1169530,\n想知道最近该不该跳槽？ \n近日，全球职场社交平台LinkedIn（领英）携手全球知名人...,"{数字经济, 人力资本, 人才流动, 企业, 第一财经, 互联网公司, 人才, 领英}",数字经济时代，中国职场人除了互联网公司还能去哪儿？,"{背景, 竞争力, 步入, 起来, 实际上, 变动, 时代, 时间, 一些, 报告, 此外,...","{去, 能, 公司, 哪儿, 中国, 除了, 经济, 职场, 还, 数字, 时代, 互联网}",,,
3,1169531,\n“没有典型场景应用没有意义，没有超越英伟达的芯片没有意义，没有世界级的算法没有意义。”依...,"{计算机视觉, 推理, 第一财经, 芯片, 算法, 依图科技, 英伟达, ai芯片, 人工智...",对标巨头英伟达，依图科技发布云端AI芯片,"{商汤, 提供, 竞争力, 这次, 体系结构, 投资, 历史, 是, 实时, 意味着, 要强...","{英, 巨头, 科技, 伟达, 发布, 芯片, 云端, 对标, 依图}",,,
4,1169532,前言爱奇艺HomeAI服务于iQIYI内的多个平台和产品，提供以视频点播为中心的智能语音交互...,"{识别, 语言模型, 语音交互, 语音识别, 模型, 爱奇艺, 语音助手}",爱奇艺HomeAI在语音交互领域的探索,"{提供, 学习, 增强, 鲁棒性, 到, 广泛应用, 转化, 是, 以外, 声临, 供, 通...","{领域, 探索, 语音, 交互, 爱奇艺}",,,
5,1169533,\n第三届世界智能大会还有10天就开幕了，16日至19日在天津梅江会展中心与大家见面的第三届...,"{智能驾驶, 应用, 大会, 自动驾驶汽车, 挑战赛, 天津, 智能, 智慧城市, 智慧生活...",第三届世界智能大会将于本月16日至19日在津举行,"{开幕, 版微, 工业, 时代, 一些, 管理局, 挑战赛, 亚太, 事项, 面向社会, 这...","{于, 举行, 日至, 大会, 津, 日, 智能, 将, 本月, 第三届, 世界}",,,
6,1169534,\n美国当地时间 4 月 29 日，知名玩具机器人 Vector 的制造商 Anki 宣布倒...,"{产品, 玩具, 语音交互, 交互, 机器人, 语音助手, 苹果, 扫地机器人}",苹果发布会上的机器人公司宣布倒闭，他们可能只是来早了8个世纪,"{研发, 周期, 显然, 却, 时间, 核心, 宠物, 一些, 量产, 可爱, 啤酒, 机器...","{世纪, 早, 发布会, 倒闭, 可能, 只是, 公司, 他们, 来, 了, 上, 机器人,...",,,
7,1169535,\n消费科技快速发展，企业必须持续变革，否则就有被淘汰的危险——初创企业尤其如此。当然，如果...,"{智能手表, 出门问问, 企业, 智能耳机, 语音助理, 独角兽, 问问魔镜, 人工智能, 谷歌}",独角兽出门问问重拾AI愿景，WHY？,"{拥有, 公共, 北京, 到, 这次, 能够, 是, 不足, 正, 同样, 个人, 将, 专...","{出门, 愿景, 重拾, 独角兽, 问问}",,,
8,1169536,"在位于德州达拉斯以北约 40 公里的小城弗里斯克（Frisco, TX），一辆行驶在路面上的...","{吴恩达, 打车, 王弢, 司机, 无人驾驶, 车辆, 自动驾驶, 车}",Drive.ai无人打车服务进德州，我们用15分钟见识了这位「新手驾驶员」,"{公共, 小时, 的确, 西部, 接送, 时间, 紧急制动, 一些, 路上, 开始, 五星,...","{我们, 新手, 无人, 驾驶员, 」, 了, 打车, 分钟, 「, 德州, 见识, 进, ...",,,
9,1169537,上个月，微软代季峰等研究者发布的一篇论文提出了一种可变形卷积网络，该研究「引入了两种新的模块...,"{可变形卷积网络, 机器之心, 代码}",微软发布可变形卷积网络代码：可用于多种复杂视觉任务,"{之, 这篇, 提供, 脚本, 内部, 到, 其仅, 保存, 看起来, 专栏, 该项, 组,...","{复杂, 卷积, 可, 发布, 微软, 代码, 视觉, 网络, 变形, 任务, 多种, 用于}",,,


In [163]:
taged(ch1,'name','tag_name')

,id,content,name,title,ccontent,ctitle,tag_title,tag_name,tag_content
0,1169528,\n利用视觉辅助仪器读报纸。(图片来源：日本《朝日新闻》网站) \n人民网东京5月6日电 据...,"{识别, ai技术, 陈建军, 视觉, 人脸, 日本, 摄像机}",日本推出一款视觉辅助仪器 利用AI技术可将文字转化为声音,"{报纸, 到, 审稿, 福利, 转化, 福祉, 是, 体验, 这款, 你, 知道, 传达, ...","{声音, 一款, 仪器, 利用, 将, 转化, 可, 视觉, 技术, 推出, 辅助, 文字,...",,{},
1,1169529,虽然遗传学和现代影像学已经取得重大进展，但诊断结果还是会令大部分乳腺癌患者感到震惊。对一些患...,"{混淆矩阵, 乳腺癌, 深度学习, 模型, 电子病历}",早发现早治疗系列之乳腺癌：DL模型为患者争取五年治疗时间,"{×, 混合, 同样, 供应商, 起来, 分配, 最低, 太迟, 一些, 重要性, 此外, ...","{之, 时间, 早, 乳腺癌, 争取, 五年, 治疗, 模型, 系列, 发现, 患者}",,{深度学习},
2,1169530,\n想知道最近该不该跳槽？ \n近日，全球职场社交平台LinkedIn（领英）携手全球知名人...,"{数字经济, 人力资本, 人才流动, 企业, 第一财经, 互联网公司, 人才, 领英}",数字经济时代，中国职场人除了互联网公司还能去哪儿？,"{背景, 竞争力, 步入, 起来, 实际上, 变动, 时代, 时间, 一些, 报告, 此外,...","{去, 能, 公司, 哪儿, 中国, 除了, 经济, 职场, 还, 数字, 时代, 互联网}",,{},
3,1169531,\n“没有典型场景应用没有意义，没有超越英伟达的芯片没有意义，没有世界级的算法没有意义。”依...,"{计算机视觉, 推理, 第一财经, 芯片, 算法, 依图科技, 英伟达, ai芯片, 人工智...",对标巨头英伟达，依图科技发布云端AI芯片,"{商汤, 提供, 竞争力, 这次, 体系结构, 投资, 历史, 是, 实时, 意味着, 要强...","{英, 巨头, 科技, 伟达, 发布, 芯片, 云端, 对标, 依图}",,{计算机视觉},
4,1169532,前言爱奇艺HomeAI服务于iQIYI内的多个平台和产品，提供以视频点播为中心的智能语音交互...,"{识别, 语言模型, 语音交互, 语音识别, 模型, 爱奇艺, 语音助手}",爱奇艺HomeAI在语音交互领域的探索,"{提供, 学习, 增强, 鲁棒性, 到, 广泛应用, 转化, 是, 以外, 声临, 供, 通...","{领域, 探索, 语音, 交互, 爱奇艺}",,{},
5,1169533,\n第三届世界智能大会还有10天就开幕了，16日至19日在天津梅江会展中心与大家见面的第三届...,"{智能驾驶, 应用, 大会, 自动驾驶汽车, 挑战赛, 天津, 智能, 智慧城市, 智慧生活...",第三届世界智能大会将于本月16日至19日在津举行,"{开幕, 版微, 工业, 时代, 一些, 管理局, 挑战赛, 亚太, 事项, 面向社会, 这...","{于, 举行, 日至, 大会, 津, 日, 智能, 将, 本月, 第三届, 世界}",,{通用人工智能},
6,1169534,\n美国当地时间 4 月 29 日，知名玩具机器人 Vector 的制造商 Anki 宣布倒...,"{产品, 玩具, 语音交互, 交互, 机器人, 语音助手, 苹果, 扫地机器人}",苹果发布会上的机器人公司宣布倒闭，他们可能只是来早了8个世纪,"{研发, 周期, 显然, 却, 时间, 核心, 宠物, 一些, 量产, 可爱, 啤酒, 机器...","{世纪, 早, 发布会, 倒闭, 可能, 只是, 公司, 他们, 来, 了, 上, 机器人,...",,{},
7,1169535,\n消费科技快速发展，企业必须持续变革，否则就有被淘汰的危险——初创企业尤其如此。当然，如果...,"{智能手表, 出门问问, 企业, 智能耳机, 语音助理, 独角兽, 问问魔镜, 人工智能, 谷歌}",独角兽出门问问重拾AI愿景，WHY？,"{拥有, 公共, 北京, 到, 这次, 能够, 是, 不足, 正, 同样, 个人, 将, 专...","{出门, 愿景, 重拾, 独角兽, 问问}",,{},
8,1169536,"在位于德州达拉斯以北约 40 公里的小城弗里斯克（Frisco, TX），一辆行驶在路面上的...","{吴恩达, 打车, 王弢, 司机, 无人驾驶, 车辆, 自动驾驶, 车}",Drive.ai无人打车服务进德州，我们用15分钟见识了这位「新手驾驶员」,"{公共, 小时, 的确, 西部, 接送, 时间, 紧急制动, 一些, 路上, 开始, 五星,...","{我们, 新手, 无人, 驾驶员, 」, 了, 打车, 分钟, 「, 德州, 见识, 进, ...",,{},
9,1169537,上个月，微软代季峰等研究者发布的一篇论文提出了一种可变形卷积网络，该研究「引入了两种新的模块...,"{可变形卷积网络, 机器之心, 代码}",微软发布可变形卷积网络代码：可用于多种复杂视觉任务,"{之, 这篇, 提供, 脚本, 内部, 到, 其仅, 保存, 看起来, 专栏, 该项, 组,...","{复杂, 卷积, 可, 发布, 微软, 代码, 视觉, 网络, 变形, 任务, 多种, 用于}",,{},


,id,content,name,title,ccontent,ctitle,tag_title,tag_name,tag_content
0,1169528,\n利用视觉辅助仪器读报纸。(图片来源：日本《朝日新闻》网站) \n人民网东京5月6日电 据...,"{识别, ai技术, 陈建军, 视觉, 人脸, 日本, 摄像机}",日本推出一款视觉辅助仪器 利用AI技术可将文字转化为声音,"{报纸, 到, 审稿, 福利, 转化, 福祉, 是, 体验, 这款, 你, 知道, 传达, ...","{声音, 一款, 仪器, 利用, 将, 转化, 可, 视觉, 技术, 推出, 辅助, 文字,...",,{},
1,1169529,虽然遗传学和现代影像学已经取得重大进展，但诊断结果还是会令大部分乳腺癌患者感到震惊。对一些患...,"{混淆矩阵, 乳腺癌, 深度学习, 模型, 电子病历}",早发现早治疗系列之乳腺癌：DL模型为患者争取五年治疗时间,"{×, 混合, 同样, 供应商, 起来, 分配, 最低, 太迟, 一些, 重要性, 此外, ...","{之, 时间, 早, 乳腺癌, 争取, 五年, 治疗, 模型, 系列, 发现, 患者}",,{深度学习},
2,1169530,\n想知道最近该不该跳槽？ \n近日，全球职场社交平台LinkedIn（领英）携手全球知名人...,"{数字经济, 人力资本, 人才流动, 企业, 第一财经, 互联网公司, 人才, 领英}",数字经济时代，中国职场人除了互联网公司还能去哪儿？,"{背景, 竞争力, 步入, 起来, 实际上, 变动, 时代, 时间, 一些, 报告, 此外,...","{去, 能, 公司, 哪儿, 中国, 除了, 经济, 职场, 还, 数字, 时代, 互联网}",,{},
3,1169531,\n“没有典型场景应用没有意义，没有超越英伟达的芯片没有意义，没有世界级的算法没有意义。”依...,"{计算机视觉, 推理, 第一财经, 芯片, 算法, 依图科技, 英伟达, ai芯片, 人工智...",对标巨头英伟达，依图科技发布云端AI芯片,"{商汤, 提供, 竞争力, 这次, 体系结构, 投资, 历史, 是, 实时, 意味着, 要强...","{英, 巨头, 科技, 伟达, 发布, 芯片, 云端, 对标, 依图}",,{计算机视觉},
4,1169532,前言爱奇艺HomeAI服务于iQIYI内的多个平台和产品，提供以视频点播为中心的智能语音交互...,"{识别, 语言模型, 语音交互, 语音识别, 模型, 爱奇艺, 语音助手}",爱奇艺HomeAI在语音交互领域的探索,"{提供, 学习, 增强, 鲁棒性, 到, 广泛应用, 转化, 是, 以外, 声临, 供, 通...","{领域, 探索, 语音, 交互, 爱奇艺}",,{},
5,1169533,\n第三届世界智能大会还有10天就开幕了，16日至19日在天津梅江会展中心与大家见面的第三届...,"{智能驾驶, 应用, 大会, 自动驾驶汽车, 挑战赛, 天津, 智能, 智慧城市, 智慧生活...",第三届世界智能大会将于本月16日至19日在津举行,"{开幕, 版微, 工业, 时代, 一些, 管理局, 挑战赛, 亚太, 事项, 面向社会, 这...","{于, 举行, 日至, 大会, 津, 日, 智能, 将, 本月, 第三届, 世界}",,{通用人工智能},
6,1169534,\n美国当地时间 4 月 29 日，知名玩具机器人 Vector 的制造商 Anki 宣布倒...,"{产品, 玩具, 语音交互, 交互, 机器人, 语音助手, 苹果, 扫地机器人}",苹果发布会上的机器人公司宣布倒闭，他们可能只是来早了8个世纪,"{研发, 周期, 显然, 却, 时间, 核心, 宠物, 一些, 量产, 可爱, 啤酒, 机器...","{世纪, 早, 发布会, 倒闭, 可能, 只是, 公司, 他们, 来, 了, 上, 机器人,...",,{},
7,1169535,\n消费科技快速发展，企业必须持续变革，否则就有被淘汰的危险——初创企业尤其如此。当然，如果...,"{智能手表, 出门问问, 企业, 智能耳机, 语音助理, 独角兽, 问问魔镜, 人工智能, 谷歌}",独角兽出门问问重拾AI愿景，WHY？,"{拥有, 公共, 北京, 到, 这次, 能够, 是, 不足, 正, 同样, 个人, 将, 专...","{出门, 愿景, 重拾, 独角兽, 问问}",,{},
8,1169536,"在位于德州达拉斯以北约 40 公里的小城弗里斯克（Frisco, TX），一辆行驶在路面上的...","{吴恩达, 打车, 王弢, 司机, 无人驾驶, 车辆, 自动驾驶, 车}",Drive.ai无人打车服务进德州，我们用15分钟见识了这位「新手驾驶员」,"{公共, 小时, 的确, 西部, 接送, 时间, 紧急制动, 一些, 路上, 开始, 五星,...","{我们, 新手, 无人, 驾驶员, 」, 了, 打车, 分钟, 「, 德州, 见识, 进, ...",,{},
9,1169537,上个月，微软代季峰等研究者发布的一篇论文提出了一种可变形卷积网络，该研究「引入了两种新的模块...,"{可变形卷积网络, 机器之心, 代码}",微软发布可变形卷积网络代码：可用于多种复杂视觉任务,"{之, 这篇, 提供, 脚本, 内部, 到, 其仅, 保存, 看起来, 专栏, 该项, 组,...","{复杂, 卷积, 可, 发布, 微软, 代码, 视觉, 网络, 变形, 任务, 多种, 用于}",,{},
